<a href="https://colab.research.google.com/github/BabiiIn/RusExpress/blob/main/%D0%92%D1%8B%D1%8F%D0%B2%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5_%D1%81%D1%83%D1%89%D0%BD%D0%BE%D1%81%D1%82%D0%B5%D0%B9_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ipython-autotime # Подсчет времени выполнения операций
%load_ext autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.3 MB/s eta 0:00:00
time: 342 µs (started: 2024-03-04 08:12:33 +00:00)


# Установка библиотек для чтения файлов

In [ ]:
!pip install langchain_community # 0.0.24
!pip install unstructured # 0.12.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.4/252.4 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 43.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 45.3 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=d8174b035f5c887942727d1e4c39b3aa35bdd5b456bc7d55efbf9e9

# Загрузка файлов

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
time: 23.7 s (started: 2024-03-04 08:13:30 +00:00)


In [ ]:
import os
import zipfile

time: 622 µs (started: 2024-03-04 08:13:57 +00:00)


In [ ]:
def load_and_extract(zip_file, extract_to):

    if not os.path.exists(extract_to):
        os.makedirs(extract_to)

    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

time: 731 µs (started: 2024-03-04 08:14:01 +00:00)


In [ ]:
zip_file = '/content/drive/MyDrive/RusExpress/1.zip'
extract_to = 'docs'

load_and_extract(zip_file, extract_to)

time: 917 ms (started: 2024-03-04 08:14:27 +00:00)


In [ ]:
folder_path = os.path.join("./", extract_to)

all_files = sorted(os.listdir(folder_path))
doc_files = [fn for fn in all_files if fn.lower().endswith('.doc') or fn.lower().endswith('.docx')]
xls_files = [fn for fn in all_files if fn.lower().endswith('.xls') or fn.lower().endswith('.xlsx')]
pdf_files = [fn for fn in all_files if fn.lower().endswith('.pdf')]

time: 2.24 ms (started: 2024-03-04 08:14:41 +00:00)


In [ ]:
print(doc_files)

['1087300000923000020_0.docx', '2019500000123000291_0.docx', '2025500000123000013_0.docx', '9800600000423000056_0.docx']
time: 610 µs (started: 2024-03-04 08:16:01 +00:00)


# doc

In [ ]:
!pip install docx2txt
!pip install python-docx

  Preparing metadata (setup.py) ... done
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3960 sha256=067c8f2ebfdee7db20491ddc4155b13a1d8ce08588fa3c595a6bf56c7507bbc8
  Stored in directory: /root/.cache/pip/wheels/22/58/cf/093d0a6c3ecfdfc5f6ddd5524043b88e59a9a199cb02352966
Successfully built docx2txt
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 2.1 MB/s eta 0:00:00
time: 15.1 s (started: 2024-03-04 08:16:08 +00:00)


## Чтение файлов

### Docx2txtLoader

In [ ]:
from langchain_community.document_loaders import Docx2txtLoader

time: 1.43 s (started: 2024-03-04 08:18:09 +00:00)


In [ ]:
all_doc_documents = []
not_correct_files = 0

for fn in doc_files[:10]:
    try:
        doc_document = Docx2txtLoader(os.path.join(folder_path, fn)).load()
        all_doc_documents.extend(doc_document)
    except:
        print(f'Файл {fn} имеет не правильный формат.')
        not_correct_files += 1

print(f'Не правильных файлов {not_correct_files}')

Не правильных файлов 0
time: 297 ms (started: 2024-03-04 08:18:13 +00:00)


In [ ]:
len(all_doc_documents)

4

time: 2.68 ms (started: 2024-03-04 08:18:22 +00:00)


In [ ]:
doc_files

['1087300000923000020_0.docx',
 '2019500000123000291_0.docx',
 '2025500000123000013_0.docx',
 '9800600000423000056_0.docx']

time: 3.4 ms (started: 2024-03-04 08:18:29 +00:00)


### UnstructuredWordDocumentLoader

In [ ]:
from langchain_community.document_loaders import UnstructuredWordDocumentLoader

time: 407 µs (started: 2024-03-04 08:19:01 +00:00)


In [ ]:
# doc_file = '/content/docx/0101100009423000050.docx'
# loader = UnstructuredWordDocumentLoader(doc_file)
# data = loader.load()

In [ ]:
all_doc_documents = []
not_correct_files = 0

for fn in doc_files[:10]:
    try:
        doc_document = UnstructuredWordDocumentLoader(os.path.join(folder_path, fn)).load()
        all_doc_documents.extend(doc_document)
    except:
        print(f'Файл {fn} имеет не правильный формат.')
        not_correct_files += 1

print(f'Не правильных файлов {not_correct_files}')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Не правильных файлов 0
time: 10.3 s (started: 2024-03-04 08:19:09 +00:00)


In [ ]:
len(all_doc_documents)

4

time: 3.54 ms (started: 2024-03-04 08:19:24 +00:00)


In [ ]:
all_doc_documents[0].page_content

'\t\t\t                                     СПЕЦИФИКАЦИЯ\n\nПОСТАВЛЯЕМЫХ ТОВАРОВ\n\n№ Наименование товара Размер Ед.\nизм. Количество Цена с НДС\nРУБ Сумма с НДС\nРУБ 1 Футболка х/б   белая Унисекс плотность материала не менее 160 г/м2 тип полотна- кулирная гладь не менее 92% хлопок не менее 8 % эластан предназначенная для нанесения рисунка методом шелкографии упаковка индивидуальная 42 шт 5 2 46 шт 20 3 48 шт 30 4 50 шт 30 5 52 шт 30 6 54 шт 15 7 56 шт 10 8 60 шт 7 9 Футболка х/б   васильковая Унисекс плотность материала не менее 160 г/м2 тип полотна- кулирная гладь не менее 92% хлопок не менее 8 % эластан предназначенная для нанесения рисунка методом шелкографии упаковка индивидуальная 36 шт 5 10 38 шт 5 11 40 шт 5 12 48 шт 5 13 50 шт 5 14 52 шт 5 15 56 шт 5 16 60 шт 3 17 Футболка х/б   красная Унисекс плотность материала не менее 160 г/м2 тип полотна- кулирная гладь не менее 92% хлопок не менее 8 % эластан предназначенная для нанесения рисунка методом шелкографии упаковка индивидуал

time: 3.41 ms (started: 2024-03-04 08:19:27 +00:00)


In [ ]:
all_doc_documents[0].metadata

{'source': './docs/1087300000923000020_0.docx'}

time: 3.54 ms (started: 2024-03-04 08:19:31 +00:00)


## Извлечение сущностей

### Загрузка библиотек

In [ ]:
!pip install langchain openai langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.2/806.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.5 MB/s eta 0:00:00
time: 10.7 s (started: 2024-03-04 08:19:36 +00:00)


In [ ]:
import os
import openai
import getpass

time: 433 ms (started: 2024-03-04 08:19:51 +00:00)


In [ ]:
openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

OpenAI API Key:··········
time: 23.8 s (started: 2024-03-04 08:19:53 +00:00)


### create_extraction_chain

In [ ]:
from langchain.chains import create_extraction_chain
from langchain_openai import ChatOpenAI
import time
import pandas as pd

time: 1.5 s (started: 2024-03-04 08:20:21 +00:00)


In [ ]:
schema = {
          "properties": {
              "Товар": {"type": "string"},
              "Год изготовления": {"type": "string"},
              "Гарантийный срок": {"type": "string"},
              "Количество": {"type": "string"},
                        },
          "required": ["Товар", "Год изготовления", "Гарантийный срок", "Количество"],
         }


time: 711 µs (started: 2024-03-04 08:20:25 +00:00)


In [ ]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")
chain = create_extraction_chain(schema, llm)
extract_list = []
for n, document in enumerate(all_doc_documents):
    try:
        time.sleep(20)
        extract_text = chain.invoke(document.page_content)
        for i in range(len(extract_text['text'])):
            extract_text['text'][i].update(document.metadata)
        extract_list.append(extract_text)
    except:
        print(f'Документ {n} не распознан.')

Документ 1 не распознан.
time: 1min 42s (started: 2024-03-04 08:20:28 +00:00)


In [ ]:
line_list = [sublist[item] for sublist in extract_list for item in sublist if item == 'text']
flat_list = [item for sublist in line_list for item in sublist]
df = pd.DataFrame(flat_list)
df

,Товар,Количество,source,Год изготовления,Гарантийный срок
0,Футболка х/б белая,42 шт,./docs/1087300000923000020_0.docx,NaN,NaN
1,Футболка х/б васильковая,36 шт,./docs/1087300000923000020_0.docx,NaN,NaN
2,Футболка х/б красная,60 шт,./docs/1087300000923000020_0.docx,NaN,NaN
3,Футболка х/б черная,44 шт,./docs/1087300000923000020_0.docx,NaN,NaN
4,Футболка х/б зеленая (темная),42 шт,./docs/1087300000923000020_0.docx,NaN,NaN
5,Бейсболка велюр белая,ONESIZE шт,./docs/1087300000923000020_0.docx,NaN,NaN
6,Бейсболка велюр красная (темная),ONESIZE шт,./docs/1087300000923000020_0.docx,NaN,NaN
7,Бейсболка велюр синяя (васильковая),ONESIZE шт,./docs/1087300000923000020_0.docx,NaN,NaN
8,Бейсболка велюр зеленая (темная),ONESIZE шт,./docs/1087300000923000020_0.docx,NaN,NaN
9,Объекты капитального строительства,,./docs/2025500000123000013_0.docx,,


time: 38 ms (started: 2024-03-04 08:22:26 +00:00)


### Черновики

In [ ]:
extract_list[0]

{'input': '\t\t\t                                     СПЕЦИФИКАЦИЯ\n\nПОСТАВЛЯЕМЫХ ТОВАРОВ\n\n№ Наименование товара Размер Ед.\nизм. Количество Цена с НДС\nРУБ Сумма с НДС\nРУБ 1 Футболка х/б   белая Унисекс плотность материала не менее 160 г/м2 тип полотна- кулирная гладь не менее 92% хлопок не менее 8 % эластан предназначенная для нанесения рисунка методом шелкографии упаковка индивидуальная 42 шт 5 2 46 шт 20 3 48 шт 30 4 50 шт 30 5 52 шт 30 6 54 шт 15 7 56 шт 10 8 60 шт 7 9 Футболка х/б   васильковая Унисекс плотность материала не менее 160 г/м2 тип полотна- кулирная гладь не менее 92% хлопок не менее 8 % эластан предназначенная для нанесения рисунка методом шелкографии упаковка индивидуальная 36 шт 5 10 38 шт 5 11 40 шт 5 12 48 шт 5 13 50 шт 5 14 52 шт 5 15 56 шт 5 16 60 шт 3 17 Футболка х/б   красная Унисекс плотность материала не менее 160 г/м2 тип полотна- кулирная гладь не менее 92% хлопок не менее 8 % эластан предназначенная для нанесения рисунка методом шелкографии упаковка 

time: 5.47 ms (started: 2024-03-04 08:22:43 +00:00)


In [ ]:
extract_list[0]['text'][0].update({'source': './docs/0101100009423000050.docx'})

time: 592 µs (started: 2024-03-04 08:22:53 +00:00)


In [ ]:
extract_list[0]['text']

[{'Товар': 'Футболка х/б белая',
  'Количество': '42 шт',
  'source': './docs/0101100009423000050.docx'},
 {'Товар': 'Футболка х/б васильковая',
  'Количество': '36 шт',
  'source': './docs/1087300000923000020_0.docx'},
 {'Товар': 'Футболка х/б красная',
  'Количество': '60 шт',
  'source': './docs/1087300000923000020_0.docx'},
 {'Товар': 'Футболка х/б черная',
  'Количество': '44 шт',
  'source': './docs/1087300000923000020_0.docx'},
 {'Товар': 'Футболка х/б зеленая (темная)',
  'Количество': '42 шт',
  'source': './docs/1087300000923000020_0.docx'},
 {'Товар': 'Бейсболка велюр белая',
  'Количество': 'ONESIZE шт',
  'source': './docs/1087300000923000020_0.docx'},
 {'Товар': 'Бейсболка велюр красная (темная)',
  'Количество': 'ONESIZE шт',
  'source': './docs/1087300000923000020_0.docx'},
 {'Товар': 'Бейсболка велюр синяя (васильковая)',
  'Количество': 'ONESIZE шт',
  'source': './docs/1087300000923000020_0.docx'},
 {'Товар': 'Бейсболка велюр зеленая (темная)',
  'Количество': 'ONESI

time: 5.47 ms (started: 2024-03-04 08:22:56 +00:00)


In [ ]:
df = pd.DataFrame(extract_list[0]['text'])
df

,Товар,Количество,source
0,Футболка х/б белая,42 шт,./docs/0101100009423000050.docx
1,Футболка х/б васильковая,36 шт,./docs/1087300000923000020_0.docx
2,Футболка х/б красная,60 шт,./docs/1087300000923000020_0.docx
3,Футболка х/б черная,44 шт,./docs/1087300000923000020_0.docx
4,Футболка х/б зеленая (темная),42 шт,./docs/1087300000923000020_0.docx
5,Бейсболка велюр белая,ONESIZE шт,./docs/1087300000923000020_0.docx
6,Бейсболка велюр красная (темная),ONESIZE шт,./docs/1087300000923000020_0.docx
7,Бейсболка велюр синяя (васильковая),ONESIZE шт,./docs/1087300000923000020_0.docx
8,Бейсболка велюр зеленая (темная),ONESIZE шт,./docs/1087300000923000020_0.docx


time: 19 ms (started: 2024-03-04 08:23:01 +00:00)


In [ ]:
flat_list = []
for sublist in extract_list:
    for item in sublist:
        if item == 'text':
            print(sublist[item])
            flat_list.append(sublist[item])

[{'Товар': 'Футболка х/б белая', 'Количество': '42 шт', 'source': './docs/0101100009423000050.docx'}, {'Товар': 'Футболка х/б васильковая', 'Количество': '36 шт', 'source': './docs/1087300000923000020_0.docx'}, {'Товар': 'Футболка х/б красная', 'Количество': '60 шт', 'source': './docs/1087300000923000020_0.docx'}, {'Товар': 'Футболка х/б черная', 'Количество': '44 шт', 'source': './docs/1087300000923000020_0.docx'}, {'Товар': 'Футболка х/б зеленая (темная)', 'Количество': '42 шт', 'source': './docs/1087300000923000020_0.docx'}, {'Товар': 'Бейсболка велюр белая', 'Количество': 'ONESIZE шт', 'source': './docs/1087300000923000020_0.docx'}, {'Товар': 'Бейсболка велюр красная (темная)', 'Количество': 'ONESIZE шт', 'source': './docs/1087300000923000020_0.docx'}, {'Товар': 'Бейсболка велюр синяя (васильковая)', 'Количество': 'ONESIZE шт', 'source': './docs/1087300000923000020_0.docx'}, {'Товар': 'Бейсболка велюр зеленая (темная)', 'Количество': 'ONESIZE шт', 'source': './docs/1087300000923000

In [ ]:
len(flat_list)

3

time: 3.06 ms (started: 2024-03-04 08:23:14 +00:00)


In [ ]:
flat_list = []
for sublist in extract_list:
    for item in sublist:
        if item == 'text':
            print(item)
            flat_list.append(item)

text
text
text
time: 2.26 ms (started: 2024-03-04 08:23:17 +00:00)


### 2 с помощью pydantic

In [ ]:
from typing import Optional
from langchain.chains import create_extraction_chain_pydantic
from langchain_core.pydantic_v1 import BaseModel

time: 835 µs (started: 2024-03-04 08:23:21 +00:00)


In [ ]:
class Properties(BaseModel):
    name: str

time: 1.62 ms (started: 2024-03-04 08:23:23 +00:00)


In [ ]:
chain = create_extraction_chain_pydantic(pydantic_schema=Properties, llm=llm)

time: 2.2 ms (started: 2024-03-04 08:23:26 +00:00)


In [ ]:
inp = data[0].page_content
chain.run(inp)

NameError: name 'data' is not defined

time: 390 ms (started: 2024-03-04 08:23:28 +00:00)


# xls

## Чтение файлов

In [ ]:
zip_file = '/content/drive/MyDrive/Intern/RusExpress/xlsx.zip'
extract_to = 'xlsx'

def load_and_extract(zip_file, extract_to):

### UnstructuredExcelLoader

In [ ]:
from langchain_community.document_loaders import UnstructuredExcelLoader

time: 486 µs (started: 2024-03-04 08:28:19 +00:00)


In [ ]:
loader = UnstructuredExcelLoader('/content/docs/2.xlsx')
docs = loader.load()
docs[0]

Document(page_content='УТВЕРЖДАЮ:                                                                                                                  Государственное бюджетное учреждение Краснодарского края                                     «Центр обеспечения мероприятий гражданской обороны и                                       чрезвычайных ситуаций «Защита»                                                            _______________________ Д.А. Коваленко                                                        "_____" _______________________ 2022 г.                                                                           \n\nОПИСАНИЕ ОБЪЕКТА ЗАКУПКИ\n\n               (поставка товаров, оборудования без комплектности)\n\nПоставка жилого полевого комплекса для создания резервов материальных ресурсов для ликвидации чрезвычайных ситуаций межмуниципального и регионального характера\n\n(указывается предмет контракта)\n\n\n\n\n№ п/п\nНаименование товара, входящего в объект закупки\nКод по ОКП

time: 384 ms (started: 2024-03-04 08:29:30 +00:00)


In [ ]:
loader = UnstructuredExcelLoader('/content/xlsx/0108500000423001283.XLS')
docs = loader.load()
docs[0]

Document(page_content='Приложение 1\n\nк извещению об электронном аукционе\n\n\nк извещению об электронном аукционе\n\n\nОписание объекта закупки, характеристики и показатели (параметры), позволяющие определить соответствие закупаемого товара требованиям заказчика\n\nИдентификационный код закупки: 232120600358212060100100100010000244\n\nИдентификационный код закупки: 232120600358212060100100100010000244\n\nКТРУ: 32.50.50.190-00002627\n\n\n\n\n\n\nХарактеристики из КТРУ\n\n\n\n\n№ п.п.\nНаименование показателя (характеристики)\nОбоснование требований, предъявленных заказчиком\nЗначение показателя (характеристики), установленное заказчиком\n\n\nХарактеристики из КТРУ\n\n\n\n\n\n1\nВысота, мм\nХарактеристика из КТРУ\n≥ 1045 и ≤ 2300\n\n\n2\nГлубина, мм\nХарактеристика из КТРУ\n≥ 600 и ≤ 844\n\n\n3\nШирина, мм\nХарактеристика из КТРУ\n≥ 994 и ≤ 2420\n\n\n4\nЗащитные крышки на розетках\nХарактеристика из КТРУ\nДа\n\n\n5\nКоличество рабочих камер, штука\nХарактеристика из КТРУ\n1\n\n\n6\nОтк

## Извлечение сущностей

### Загрузка библиотек

In [ ]:
!pip install langchain openai langchain-openai

time: 7.94 s (started: 2024-03-04 08:29:46 +00:00)


In [ ]:
import os
import openai
import getpass

time: 439 µs (started: 2024-03-04 08:29:56 +00:00)


In [ ]:
openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

OpenAI API Key:··········
time: 9.15 s (started: 2024-03-04 08:29:58 +00:00)


### create_extraction_chain

In [ ]:
from langchain.chains import create_extraction_chain
from langchain_openai import ChatOpenAI

time: 619 µs (started: 2024-03-04 08:30:12 +00:00)


In [ ]:
# Schema
schema = {
    "properties": {
        "name": {"type": "string"},
        "height": {"type": "integer"},
                  },
    "required": ["name", "height"],
         }

# Input
inp = docs[0].page_content

# Run chain
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")
chain = create_extraction_chain(schema, llm)
chain.run(inp)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


[{'name': 'Жилой полевой комплекс', 'height': 183}]

time: 1.24 s (started: 2024-03-04 08:30:15 +00:00)


### 2 с помощью pydantic

In [ ]:
from typing import Optional
from langchain.chains import create_extraction_chain_pydantic
from langchain_core.pydantic_v1 import BaseModel

time: 738 µs (started: 2024-03-04 08:30:24 +00:00)


In [ ]:
class Properties(BaseModel):
    name: str

time: 1.49 ms (started: 2024-03-04 08:30:27 +00:00)


In [ ]:
chain = create_extraction_chain_pydantic(pydantic_schema=Properties, llm=llm)

time: 2.73 ms (started: 2024-03-04 08:30:29 +00:00)


In [ ]:
inp = docs[0].page_content
chain.run(inp)

[Properties(name='Жилой полевой комплекс')]

time: 1.11 s (started: 2024-03-04 08:30:31 +00:00)


# pdf

## Чтение файлов

In [ ]:
zip_file = '/content/drive/MyDrive/Intern/RusExpress/pdf.zip'
extract_to = 'pdf'

load_and_extract(zip_file, extract_to)

### PyPDFLoader

In [ ]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 6.2 MB/s eta 0:00:00


In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader('/content/pdf/0111300005122000533.pdf')
pages = loader.load_and_split()
pages[0]

Document(page_content='Описание объекта закупки  \nна поставку спортивного инвентаря\n1. Наименование организации: МБОУ «Средняя общеобразовательная школа Ж  1  с углубленным изучением отдельных предметов» НМР РТ\n2. Предмет контракта: поставка спортивного инвентаря\n3. Место поставки: 423570, РТ, г. Нижнекамск, пр. Мира д. 16\n4. Сроки поставки: В течение 1  5 календарных дней с момента заключения контракта\n5. Наименование, характеристики, количество товара:\n№\nп/пНаименование товара ХарактеристикаЕд.\nизм.Кол-\nво\nТурник разнохватовый \nОКПД 2: 32.30.14.119,  \nКТРУ: 32.30.14.119-  \n00000074\n[Способ хвата: классический;\nСпособ установки: навесное;\nРабочая длина перекладины (Миллиметр): < 1200;\nНаличие функции регулировки по высоте: нет;\nМинимальная высота перекладины (Миллиметр): > 900; \nМаксимальная высота перекладины (Миллиметр): < 1500; \nКомплектность: неопреновые ручки; стойка для пресса; брусья; \nМаксимальная масса пользователя (Килограмм): < 120штука\nОбруч гимнасти

### UnstructuredPDFLoader

In [ ]:
!pip install pdf2image
!pip install pdfminer.six
!pip install pillow-heif
!pip install unstructured-inference
!pip install pikepdf

In [ ]:
from langchain_community.document_loaders import UnstructuredPDFLoader

In [ ]:
loader = UnstructuredPDFLoader('/content/pdf/0111300005122000533.pdf')

data = loader.load()

data[0]

Document(page_content='О писание объекта закупки на поставку спортивного инвентаря\n\n1. Н а и м ен о в а н и е организации: МБОУ «Средняя общеобразовательная школа Ж 1 с углубленным изучением отдельных предметов» НМР РТ 2. П р едм ет контракта: поставка спортивного инвентаря 3. М есто поставки: 423570, РТ, г. Нижнекамск, пр. М ира д. 16 4. Сроки поставки: В течение 1 5 календарных дней с момента заключения контракта 5. Н аи м ен ов а н и е, характеристики, количество товара: № п/п\n\nН а и м ен о в а н и е товара\n\nХарактеристика\n\nТ у р н и к разнохватовый О К П Д 2: 32.30.14.119, К Т Р У : 32.30.14.119- 00000074\n\nСпособ хвата: классический; Способ установки: навесное; Рабочая длина перекладины (Миллиметр): < 1200; Наличие функции регулировки по высоте: нет; Минимальная высота перекладины (Миллиметр): > 900; Максимальная высота перекладины (Миллиметр): < 1500; Комплектность: неопреновые ручки; стойка для пресса; брусья; Максимальная масса пользователя (Килограмм): < 120\n\n[\n\nО

### extract_images - не работает с русским языком

In [ ]:
!pip install rapidocr-onnxruntime

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

In [ ]:
loader = PyPDFLoader('/content/pdf/0111300005122000490.pdf', extract_images=True)

pages = loader.load()

pages

[Document(page_content='HepeyeHb TOBapOB B cocTaBe o6beKTa 3aKyIIKH:\n《DH3HyecKaa KyJIbTypa》.\nPe3yJIBTaTOM HCIIOJIHeHHA KOHTpaKTa IOJIKeH 6BITE COpMHpOBaHHBI0 KJIacC-KOMIJIeKT, rOTOBBI0 K peaJIN3aHINH O6pa3OBaTeJIBHBIX IpOrpaMM IIO IpeIMeTy\nTpe6oBaHHA K Pe3yJIbTaTy HCHIOJIHeHHa KOHTpaKTa:\nTeXHHYeCKHMN W KaYeCTBeHHbIMN XapaKTepHCTHKaMN.\nJOIOJIHNTeJIBHBIe Tpe6OBaHHA O6yCJIOBJIeHBI HeOOXOJHMOCTbIOHOJIyYeHHA TOBapa CO 3HaYUMBIMH B O6pa3OBaTeJIHHOM IpOIeCCe DyHKIHOHaJIbHBIMH,\n3aKoHoJaTeJIbcTBomPoccuicKoi DeIepaunocTaHIapTH3auun:\nPeryJIHPOBaHHH,JOKyMeHTaMu,Pa3pa6aTHIBaeMbIMH H IpHMeHHeMbIMH B HaUHOHaJIbHOi cHCTeMe CTaHapTH3aIHH,IPHHATbIMH B cOOTBeTCTBHH c\nHPeIyCMOTpeHHLIX TeXHHYeCKHMH perJIaMeHTaMH,IpHHATbIMH B cOOTBeTCTBHH c 3aKOHOIaTeJILCTBOMPoccuicKOi DeIepaUUN O TeXHNyecKOM\nO6OCHOBaHHe He06XOJHMOCTH HCHOJIb3OBaHHA IOKa3aTeJIe,TPe6OBaHN,yCJIOBHbIX 06O3HaYeHNi H TepMHHOJIOrHN, OTJINyHbIX OT CTaHIapTHbIX,\nOKII2/KTPY\n32.30.14.120-\n32.30.14.120/\n32.30.14.119-\n32.30.14.119/\n32.30